In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df = df.filter(['icustay_id','icu_day','chloride_max','chloride_input_meq'])
# df = df.dropna(subset = ['chloride_max'])

# Chloride >= 110 in the second day
df_chl = df.query('icu_day == 2')
df_chl['chl_110'] = (df_chl['chloride_max'] >= 110).astype(int)

# Demographic
df = pd.read_csv('adm_demographics.csv')
df = df.set_index(['subject_id', 'hadm_id'])
df_demo = df.filter(['insurance','ethnicity','age','gender'])

# Weight
df = pd.read_csv('weight.csv')
df = df[df['icustay_id'].isin(icu)]
df_wt = df.filter(['icustay_id','day','weight'])

# GCS(Glasgow coma scale)
df = pd.read_csv('gcs_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df_gcs = df.filter(['icustay_id','day','mingcs'])

# First-day Vitals(Merge with GCS and Weight)
df = pd.read_csv('vitals_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df.filter(['day', 'heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.merge(df_wt, on = ['icustay_id','day']).merge(df_gcs, on = ['icustay_id','day'])
df_vitals = df_vitals.query('(day == 1) & (0 < heartrate_max <= 250) &(40<sysbp_min <190) &(20<diasbp_min<150) &(6<resprate_max<80) &(0<spo2_min<100)&(30<tempc_max<44)&(30<weight<500)&(3<mingcs<15)'
                           )
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])

# Medication from prescription table
df = pd.read_csv('medication.csv')
df_med = df.drop(df.columns[[0,3]], axis=1)
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)
df = df_med.merge(df, on = ['subject_id','hadm_id'])
df['taken'] = ((pd.to_datetime(df['intime']) <= pd.to_datetime(df['startdate'])) & 
               ((pd.to_datetime(df['startdate']) <= (pd.to_datetime(df['intime']) + datetime.timedelta(days=1)))))
df = df.filter(['subject_id','hadm_id','icustay_id','drug','taken'])
df_med = df.set_index(['subject_id','hadm_id','icustay_id'])
df_med = df_med.groupby(['subject_id','hadm_id','icustay_id','drug']).any()
df_med = df_med['taken'].unstack()
df_med = (df_med * 1).fillna(0)

# Intake and output
df = pd.read_csv('fluid_io.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.query('icu_day == 1')
df_fluid = df.set_index(['subject_id','hadm_id','icustay_id'])
df_fluid.head()

# Diagnosis Code(group by elixhauser)
df = pd.read_csv('elixhauser.csv')
df_icd = df.drop(df.columns[[0]], axis=1)
df_icd = df_icd.set_index('hadm_id')

# Labs
df = pd.read_csv('labs1.csv')
df = df.drop(df.columns[0], axis=1)
df = df[df['icustay_id'].isin(icu)]
df_labs = df.query('day == 1 & 0.1<lactate_max<40 & 90<sodium_max<190 & 1.5<potassium_max<15 & 1<albumin_max<6.5 & 5<bicarbonate_min<65 & 0.1<creatinine_max<47 & 66<chloride_max<170 & 18<glucose_max<2500 & 2<hemoglobin_min<21 & 5<platelet_min<1600  & 0.5<inr_max<20 & 1<bun_max<280 & 0.1<wbc_max<680 & 0.1<bilirubin_min<80 & 1.5<calcium_total_min<18 & 0.2<calcium_ionized_min<2.5 & 2<ast_sgot_max<25000 & 3<amylase_max<25000 &  1<lipase_max<25000 & 0.1<c_reactive_protein_max<300')
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])

# Interventions
df = pd.read_csv('interventions.csv')
df = df.query('day == 1')
df_int = df.drop(df.columns[[0]], axis=1)
df_int = df_int.set_index(['subject_id', 'hadm_id', 'icustay_id'])
df_int = df_int.filter(['label'])
df_int['taken'] = 1
df_int = df_int.groupby(['subject_id','hadm_id','icustay_id','label']).any()
df_int = df_int['taken'].unstack()
df_int = (df_int * 1).fillna(0)

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
demo = ['age', 'gender', 'ethnicity']
vitals = ['heartrate_max', 'sysbp_min', 'diasbp_min', 'resprate_max', 'weight', 'mingcs']
medication = ['norepinephrine']
fluid = ['fluid_net_input_ml']
diag = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 
        'chronic_pulmonary', 
       'diabetes_complicated',  'renal_failure',
       'liver_disease',  
        'coagulopathy',  'fluid_electrolyte',
       'blood_loss_anemia']
labs = ['chloride_max', 'sodium_max', 'bicarbonate_min']
interventions = ['EPAP','IPAP','LPM','MeanAirwayPressure']

In [5]:
df_demo = df_demo[demo]
df_vitals = df_vitals[vitals]
df_med = df_med[medication]
df_fluid = df_fluid[fluid]
df_icd = df_icd[diag]
df_labs = df_labs[labs]
df_int = df_int[interventions]
df_chl = df_chl[['chl_110','chloride_input_meq']]

In [6]:
df = df_demo.join(df_chl, how = 'inner') \
.join(df_fluid, how = 'inner') \
.join(df_icd, how = 'left') \
.join(df_labs, how = 'left') \
.join(df_int, how='left') \
.join(df_vitals, how='left') \
.join(df_med, how = 'left') 

df = df.fillna(0)
df

age gender ethnicity  chl_110  \
subject_id hadm_id icustay_id                                        
4          185777  294638      47.876712      F     WHITE        0   
6          107064  228232      65.983562      F     WHITE        0   
9          150750  220597      41.816438      M     OTHER        0   
11         194540  229441      50.180822      F     WHITE        0   
13         143045  263738      39.890411      F     WHITE        1   
17         194023  277042      47.484932      F     WHITE        0   
19         109235  273430      91.400000      M     WHITE        0   
21         109451  217847      87.495890      M     WHITE        0   
           111970  216859      87.882192      M     WHITE        0   
23         124321  234044      75.304110      M     WHITE        0   
           152223  227807      71.178082      M     WHITE        0   
26         197661  244882      72.052055      M     OTHER        0   
30         104557  225176      91.400000      M     OTHER        0   
31         128652  254478      72.312329      M     WHITE        0   
32         175413  295037      45.542466      M     WHITE        0   
33         176176  296681      82.446575      M     OTHER        0   
34         144319  290505      91.400000      M     WHITE        0   
36         165660  241249      72.775342      M     WHITE        0   
38         185910  248910      75.991781      M     WHITE        1   
41         101757  261027      56.671233      M     WHITE        0   
42         119203  210828      61.205479      M     OTHER        0   
43         146828  225852      32.789041      M     WHITE        0   
44         181750  291554      80.235616      M     OTHER        0   
52         190797  261857      38.147945      M     WHITE        0   
53         155385  239839      45.580822      M     OTHER        0   
55         190665  262723      64.202740      F     WHITE        0   
56         181711  275642      91.400000      F     WHITE        0   
59         104130  224440      87.150685      F     WHITE        0   
61         176332  252348      54.690411      M     WHITE        1   
           189535  217135      55.241096      M     WHITE        1   
...                                  ...    ...       ...      ...   
99836      101018  245746      52.136986      F     OTHER        1   
99837      197195  253877      25.408219      M     WHITE        0   
99847      155024  219261      75.676712      F     WHITE        0   
99862      178839  253685      59.134247      F     OTHER        0   
99863      100749  216757      42.720548      M     WHITE        0   
99864      111512  234084      83.252055      M     WHITE        0   
99868      177777  231346      20.452055      F     OTHER        0   
99870      124555  220524      43.621918      F     OTHER        0   
99881      172327  214377      77.816438      M     WHITE        0   
99883      150755  276467      73.931507      M     WHITE        0   
99893      128349  208873      88.608219      M     WHITE        0   
99894      127380  204236      60.041096      F     WHITE        1   
99897      162913  266801      53.586301      M     BLACK        0   
99899      188409  245756      87.013699      M     BLACK        0   
99901      131711  226541      68.873973      M     WHITE        0   
99913      175989  292375      71.136986      M  HISPANIC        0   
99936      107913  213906      91.400000      F     WHITE        0   
99937      129380  291004      65.720548      F  HISPANIC        0   
99939      159023  252442      26.967123      M  HISPANIC        0   
99944      185654  221067      85.536986      F     WHITE        0   
99955      108494  243255      63.232877      M     BLACK        1   
99957      148574  257127      68.095890      M     WHITE        0   
99965      101083  257338      66.334247      M     WHITE        0   
99966      167228  252173      77.536986      M     WHITE        0   
99982      112748  275958      65.304110      M

In [7]:
df['gender'] = (df['gender'] == 'F').astype(int)

In [8]:
df = pd.concat([df, pd.get_dummies(df['ethnicity'])], axis=1)
df = df.drop(['ethnicity', 'OTHER'], axis=1)
df

age  gender  chl_110  chloride_input_meq  \
subject_id hadm_id icustay_id                                                   
4          185777  294638      47.876712       1        0            0.000000   
6          107064  228232      65.983562       1        0          280.280000   
9          150750  220597      41.816438       0        0           37.730000   
11         194540  229441      50.180822       1        0           49.800000   
13         143045  263738      39.890411       1        1          104.928000   
17         194023  277042      47.484932       1        0           40.000000   
19         109235  273430      91.400000       0        0           55.300000   
21         109451  217847      87.495890       0        0           73.920000   
           111970  216859      87.882192       0        0          247.940000   
23         124321  234044      75.304110       0        0           68.493592   
           152223  227807      71.178082       0        0           20.000000   
26         197661  244882      72.052055       0        0           26.180000   
30         104557  225176      91.400000       0        0            0.000000   
31         128652  254478      72.312329       0        0           95.193200   
32         175413  295037      45.542466       0        0           76.230000   
33         176176  296681      82.446575       0        0           18.480000   
34         144319  290505      91.400000       0        0            6.428645   
36         165660  241249      72.775342       0        0          194.673189   
38         185910  248910      75.991781       0        1          190.443750   
41         101757  261027      56.671233       0        0          334.080000   
42         119203  210828      61.205479       0        0            0.000000   
43         146828  225852      32.789041       0        0          326.800000   
44         181750  291554      80.235616       0        0            0.000000   
52         190797  261857      38.147945       0        0           27.720000   
53         155385  239839      45.580822       0        0            0.000000   
55         190665  262723      64.202740       1        0           15.400000   
56         181711  275642      91.400000       1        0          175.305000   
59         104130  224440      87.150685       1        0           84.700000   
61         176332  252348      54.690411       0        1           30.800000   
           189535  217135      55.241096       0        1          110.880000   
...                                  ...     ...      ...                 ...   
99836      101018  245746      52.136986       1        1           35.677811   
99837      197195  253877      25.408219       0        0           77.000000   
99847      155024  219261      75.676712       1        0           92.360001   
99862      178839  253685      59.134247       1        0          163.547573   
99863      100749  216757      42.720548       0        0          266.574756   
99864      111512  234084      83.252055       0        0           20.758726   
99868      177777  231346      20.452055       1        0           36.725576   
99870      124555  220524      43.621918       1        0           16.170000   
99881      172327  214377      77.816438       0        0          225.382764   
99883      150755  276467      73.931507       0        0            0.000000   
99893      128349  208873      88.608219       0        0           25.554369   
99894      127380  204236      60.041096       1        1           31.843778   
99897      162913  266801      53.586301       0        0            0.000000   
99899      188409  245756      87.013699       0        0          468.388960   
99901      131711  226541      68.873973       0        0          189.093747   
99913      175989  292375      71.136986       0        0           75.000001   
99936      107913  213906      91.400000       1        0          3

## standarize all values to z-score

In [8]:
stan = pd.read_csv('standard.csv')
stan = stan.set_index('vitalid')
stan

,mean,median,std,iqr
vitalid,,,,
BICARBONATE,25.696443,25,5.034271,7
CHLORIDE,104.144561,104,6.322994,8
SODIUM,138.852473,139,5.119334,6
DiasBP,59.431493,58,14.794056,19
HeartRate,82.917012,82,16.126807,22
RespRate,19.693027,19,5.462299,7
SysBP,123.279089,121,22.821488,31
mingcs,13.662176,15,2.335564,1


In [9]:
df=df.replace({'heartrate_max': {0: stan.loc['HeartRate']['mean']},
               'sysbp_min': {0: stan.loc['SysBP']['mean']},
               'diasbp_min': {0: stan.loc['DiasBP']['mean']},
               'resprate_max': {0: stan.loc['RespRate']['mean']},
               'heartrate_max': {0: stan.loc['HeartRate']['mean']},
               'weight': {0: df_wt['weight'].mean()},
               'mingcs': {0: stan.loc['mingcs']['mean']},
               'chloride_max': {0: stan.loc['CHLORIDE']['mean']},
               'sodium_max': {0: stan.loc['SODIUM']['mean']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['mean']}
              
              }) 
df

age  gender  chl_110  fluid_net_input_ml  \
subject_id hadm_id icustay_id                                                   
4          185777  294638      47.876712       1        0         1015.000000   
6          107064  228232      65.983562       1        0         3842.500000   
9          150750  220597      41.816438       0        0         1516.133019   
13         143045  263738      39.890411       1        1         3152.827999   
17         194023  277042      47.484932       1        0         3857.600001   
21         109451  217847      87.495890       0        0           -6.575005   
           111970  216859      87.882192       0        0         2727.659997   
26         197661  244882      72.052055       0        0          685.000000   
30         104557  225176      91.400000       0        0         -646.750000   
31         128652  254478      72.312329       0        0        -1259.258337   
32         175413  295037      45.542466       0        0         3467.449997   
33         176176  296681      82.446575       0        0         4394.000000   
36         165660  241249      72.775342       0        0          543.618910   
38         185910  248910      75.991781       0        1         2014.593750   
41         101757  261027      56.671233       0        0         -425.000000   
42         119203  210828      61.205479       0        0        -1390.416648   
43         146828  225852      32.789041       0        0         1255.000000   
44         181750  291554      80.235616       0        0         -323.702500   
52         190797  261857      38.147945       0        0          189.000000   
53         155385  239839      45.580822       0        0          640.000000   
55         190665  262723      64.202740       1        0         -263.999999   
56         181711  275642      91.400000       1        0          686.000000   
59         104130  224440      87.150685       1        0         3698.000000   
61         176332  252348      54.690411       0        1         4187.000000   
           189535  217135      55.241096       0        1         2285.020348   
64         172056  232593      26.698630       1        0          922.500000   
65         143430  244776      26.580822       0        0          965.000000   
68         108329  272667      41.876712       1        0         2856.000000   
           170467  294232      41.821918       1        0         2846.000000   
71         111944  211832      36.528767       1        0        -1403.200004   
...                                  ...     ...      ...                 ...   
99797      135425  225994      57.802740       0        0         1877.707764   
99810      196261  272409      57.728767       0        0        -1863.102161   
99822      163117  220334      61.290411       0        0         2598.499983   
99823      179720  296047      81.969863       1        0        -2109.044126   
99836      101018  245746      52.136986       1        1         2888.419694   
99837      197195  253877      25.408219       0        0         1997.900003   
99847      155024  219261      75.676712       1        0        -1506.288885   
99862      178839  253685      59.134247       1        0          846.502738   
99863      100749  216757      42.720548       0        0         1797.155977   
99868      177777  231346      20.452055       1        0         1906.272189   
99881      172327  214377      77.816438       0        0         2990.531639   
99883      150755  276467      73.931507       0        0        -3267.153335   
99893      128349  208873      88.608219       0        0        -1540.153951   
99894      127380  204236      60.041096       1        1           60.341428   
99897      162913  266801      53.586301       0        0         3387.245996   
99899      188409  245756      87.013699       0        0         2994.929321   
99901      131711  226541      68.873973       0        0        -25

In [10]:
df['heartrate_max'] = (df['heartrate_max'] - stan.loc['HeartRate']['mean'])/stan.loc['HeartRate']['std']
df['sysbp_min'] = (df['sysbp_min'] - stan.loc['SysBP']['mean'])/stan.loc['SysBP']['std']
df['diasbp_min'] = (df['diasbp_min'] - stan.loc['DiasBP']['mean'])/stan.loc['DiasBP']['std']
df['resprate_max'] = (df['resprate_max'] - stan.loc['RespRate']['mean'])/stan.loc['RespRate']['std']
df['weight'] = (df['weight'] - df_wt['weight'].mean())/df_wt['weight'].std()
df['mingcs'] = (df['mingcs'] - stan.loc['mingcs']['mean'])/stan.loc['mingcs']['std']

df['chloride_max'] = (df['chloride_max'] - stan.loc['CHLORIDE']['mean'])/stan.loc['CHLORIDE']['std']
df['sodium_max'] = (df['sodium_max'] - stan.loc['SODIUM']['mean'])/stan.loc['SODIUM']['std']
df['bicarbonate_min'] = (df['bicarbonate_min'] - stan.loc['BICARBONATE']['mean'])/stan.loc['BICARBONATE']['std']


In [11]:
df['fluid_net_input_ml'] = (df['fluid_net_input_ml'] - df['fluid_net_input_ml'].mean())/ df['fluid_net_input_ml'].std()
df

age  gender  chl_110  fluid_net_input_ml  \
subject_id hadm_id icustay_id                                                   
4          185777  294638      47.876712       1        0           -0.078811   
6          107064  228232      65.983562       1        0            1.364459   
9          150750  220597      41.816438       0        0            0.176988   
13         143045  263738      39.890411       1        1            1.012423   
17         194023  277042      47.484932       1        0            1.372167   
21         109451  217847      87.495890       0        0           -0.600264   
           111970  216859      87.882192       0        0            0.795400   
26         197661  244882      72.052055       0        0           -0.247256   
30         104557  225176      91.400000       0        0           -0.927035   
31         128652  254478      72.312329       0        0           -1.239684   
32         175413  295037      45.542466       0        0            1.173019   
33         176176  296681      82.446575       0        0            1.645967   
36         165660  241249      72.775342       0        0           -0.319423   
38         185910  248910      75.991781       0        1            0.431422   
41         101757  261027      56.671233       0        0           -0.813845   
42         119203  210828      61.205479       0        0           -1.306633   
43         146828  225852      32.789041       0        0            0.043695   
44         181750  291554      80.235616       0        0           -0.762139   
52         190797  261857      38.147945       0        0           -0.500435   
53         155385  239839      45.580822       0        0           -0.270226   
55         190665  262723      64.202740       1        0           -0.731664   
56         181711  275642      91.400000       1        0           -0.246746   
59         104130  224440      87.150685       1        0            1.290701   
61         176332  252348      54.690411       0        1            1.540306   
           189535  217135      55.241096       0        1            0.569459   
64         172056  232593      26.698630       1        0           -0.126027   
65         143430  244776      26.580822       0        0           -0.104333   
68         108329  272667      41.876712       1        0            0.860910   
           170467  294232      41.821918       1        0            0.855805   
71         111944  211832      36.528767       1        0           -1.313158   
...                                  ...     ...      ...                 ...   
99797      135425  225994      57.802740       0        0            0.361550   
99810      196261  272409      57.728767       0        0           -1.547910   
99822      163117  220334      61.290411       0        0            0.729471   
99823      179720  296047      81.969863       1        0           -1.673449   
99836      101018  245746      52.136986       1        1            0.877458   
99837      197195  253877      25.408219       0        0            0.422901   
99847      155024  219261      75.676712       1        0           -1.365778   
99862      178839  253685      59.134247       1        0           -0.164819   
99863      100749  216757      42.720548       0        0            0.320433   
99868      177777  231346      20.452055       1        0            0.376130   
99881      172327  214377      77.816438       0        0            0.929580   
99883      150755  276467      73.931507       0        0           -2.264595   
99893      128349  208873      88.608219       0        0           -1.383065   
99894      127380  204236      60.041096       1        1           -0.566107   
99897      162913  266801      53.586301       0        0            1.132079   
99899      188409  245756      87.013699       0        0            0.931825   
99901      131711  226541      68.873973       0        0           

In [12]:
df.to_csv('data.csv')

# Normalization

## impute with median

In [20]:
df1 =df.replace({'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'sysbp_min': {0: stan.loc['SysBP']['median']},
               'diasbp_min': {0: stan.loc['DiasBP']['median']},
               'resprate_max': {0: stan.loc['RespRate']['median']},
               'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'weight': {0: df_wt['weight'].median()},
               'mingcs': {0: stan.loc['mingcs']['median']},
               'chloride_max': {0: stan.loc['CHLORIDE']['median']},
               'sodium_max': {0: stan.loc['SODIUM']['median']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['median']}
              
              }) 
df1.head()

,,,age,gender,chl_110,fluid_net_input_ml,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,...,sysbp_min,diasbp_min,resprate_max,weight,mingcs,norepinephrine,ASIAN,BLACK,HISPANIC,WHITE
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
4,185777,294638,47.876712,1,0,1015.000000,0.0,0.0,0.0,0.0,0.0,0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1
6,107064,228232,65.983562,1,0,3842.500000,0.0,0.0,0.0,0.0,0.0,0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1
9,150750,220597,41.816438,0,0,1516.133019,1.0,0.0,0.0,0.0,0.0,1,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,0
13,143045,263738,39.890411,1,1,3152.827999,0.0,0.0,0.0,0.0,0.0,1,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1
17,194023,277042,47.484932,1,0,3857.600001,0.0,0.0,0.0,0.0,0.0,0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1


In [11]:
df1.to_csv('data_original_median.csv')

## normalize

In [21]:
df1['heartrate_max'] = (df1['heartrate_max'] - stan.loc['HeartRate']['median'])/stan.loc['HeartRate']['iqr']
df1['sysbp_min'] = (df1['sysbp_min'] - stan.loc['SysBP']['median'])/stan.loc['SysBP']['iqr']
df1['diasbp_min'] = (df1['diasbp_min'] - stan.loc['DiasBP']['median'])/stan.loc['DiasBP']['iqr']
df1['resprate_max'] = (df1['resprate_max'] - stan.loc['RespRate']['median'])/stan.loc['RespRate']['iqr']
df1['weight'] = (df1['weight'] - df_wt['weight'].median())/(df_wt['weight'].quantile(0.75) - df_wt['weight'].quantile(0.25))
df1['mingcs'] = (df1['mingcs'] - stan.loc['mingcs']['median'])/stan.loc['mingcs']['iqr']

df1['chloride_max'] = (df1['chloride_max'] - stan.loc['CHLORIDE']['median'])/stan.loc['CHLORIDE']['iqr']
df1['sodium_max'] = (df1['sodium_max'] - stan.loc['SODIUM']['median'])/stan.loc['SODIUM']['iqr']
df1['bicarbonate_min'] = (df1['bicarbonate_min'] - stan.loc['BICARBONATE']['median'])/stan.loc['BICARBONATE']['iqr']

In [27]:
print(df1['age'].median(), (df1['age'].quantile(0.75) - df1['age'].quantile(0.25)),df1['age'].mean(),df1['age'].std() )

66.6712328767123 24.66575342465749 64.95149518393697 17.048611163983235


In [37]:
print(df_wt['weight'].median(), (df_wt['weight'].quantile(0.75) - df_wt['weight'].quantile(0.25)),df_wt['weight'].mean(),df_wt['weight'].std() )

80.0 28.0 83.06121690261101 24.71410967312458


In [22]:
df1['fluid_net_input_ml'] = (df1['fluid_net_input_ml'] - df1['fluid_net_input_ml'].median())/ (df1['fluid_net_input_ml'].quantile(0.75) - df1['fluid_net_input_ml'].quantile(0.25))
df1['age'] = (df1['age'] - df1['age'].median())/ (df1['age'].quantile(0.75) - df1['age'].quantile(0.25))

df1

age  gender  chl_110  fluid_net_input_ml  \
subject_id hadm_id icustay_id                                                  
4          185777  294638     -0.761968       1        0           -0.036410   
6          107064  228232     -0.027880       1        0            0.977575   
9          150750  220597     -1.007664       0        0            0.143304   
13         143045  263738     -1.085749       1        1            0.730248   
17         194023  277042     -0.777852       1        0            0.982990   
21         109451  217847      0.844274       0        0           -0.402763   
           111970  216859      0.859936       0        0            0.577776   
26         197661  244882      0.218150       0        0           -0.154753   
30         104557  225176      1.002555       0        0           -0.632339   
31         128652  254478      0.228702       0        0           -0.851994   
32         175413  295037     -0.856603       0        0            0.843076   
33         176176  296681      0.639565       0        0            1.175351   
36         165660  241249      0.247473       0        0           -0.205455   
38         185910  248910      0.377874       0        1            0.322060   
41         101757  261027     -0.405420       0        0           -0.552816   
42         119203  210828     -0.221593       0        0           -0.899030   
43         146828  225852     -1.373653       0        0            0.049657   
44         181750  291554      0.549928       0        0           -0.516490   
52         190797  261857     -1.156392       0        0           -0.332627   
53         155385  239839     -0.855048       0        0           -0.170891   
55         190665  262723     -0.100078       1        0           -0.495079   
56         181711  275642      1.002555       1        0           -0.154395   
59         104130  224440      0.830279       1        0            0.925755   
61         176332  252348     -0.485727       0        1            1.101118   
           189535  217135     -0.463401       0        1            0.419039   
64         172056  232593     -1.620571       1        0           -0.069582   
65         143430  244776     -1.625347       0        0           -0.054341   
68         108329  272667     -1.005220       1        0            0.623801   
           170467  294232     -1.007442       1        0            0.620215   
71         111944  211832     -1.222037       1        0           -0.903614   
...                                 ...     ...      ...                 ...   
99797      135425  225994     -0.359547       0        0            0.272970   
99810      196261  272409     -0.362546       0        0           -1.068542   
99822      163117  220334     -0.218150       0        0            0.531457   
99823      179720  296047      0.620238       1        0           -1.156741   
99836      101018  245746     -0.589248       1        1            0.635427   
99837      197195  253877     -1.672887       0        0            0.316073   
99847      155024  219261      0.365101       1        0           -0.940583   
99862      178839  253685     -0.305565       1        0           -0.096836   
99863      100749  216757     -0.971010       0        0            0.244083   
99868      177777  231346     -1.873820       1        0            0.283214   
99881      172327  214377      0.451849       0        0            0.672046   
99883      150755  276467      0.294346       0        0           -1.572057   
99893      128349  208873      0.889370       0        0           -0.952728   
99894      127380  204236     -0.268799       1        1           -0.378766   
99897      162913  266801     -0.530490       0        0            0.814314   
99899      188409  245756      0.824725       0        0            0.673623   
99901      131711  226541      0.089304       0        0           -1.312767   
99913      175989  292375      0.181

In [23]:
df1.to_csv('data_normalize_median.csv')